# Importation des bibliothèques

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import random
import time
import pandas as pd

# Ouverture du driver

In [ ]:
# Initialiser le WebDriver
driver = webdriver.Chrome()  # Utilisez le WebDriver approprié pour votre navigateur

# Ouvrir la page du jeu
driver.get('https://www.instagram.com/')

Connectez vous à votre compte instagram

In [ ]:
driver.get('https://www.instagram.com/instagram/')
time.sleep(2)

# Récupération des données
(allongez les durées des time.sleep() en fonction de la qualité de votre connection)

In [ ]:
posts = []
last_height = driver.execute_script("return document.body.scrollHeight")
scroll_attempts = 0
for _ in range(10):
# while True:
    # Collecte des posts visibles
    path = "//div[contains(@class, '_ac7v')]//a[contains(@role, 'link')]"
    all_posts = driver.find_elements(By.XPATH, path)
    
    for post in all_posts:
        post_link = post.get_attribute('href')
        if not any(post_link in collected_post for collected_post in posts):  # Pour éviter les doublons
            post.click()
            time.sleep(1)  # Laisser le temps au modal de se charger

            # Collecter les données du post
            try:
                date = driver.find_element(By.XPATH, '//time[@class="x1p4m5qa"]').get_attribute('datetime')
                likes = driver.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/span/a/span/span').text
                comments = driver.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/div[1]/li/div/div/div[2]/div[1]/h1').text
                
                posts.append((date, likes, comments, post_link))
            
                # Fermer le modal du post
                close_path = "/html/body/div[8]/div[1]/div/div[2]/div"
                driver.find_element(By.XPATH, close_path).click()
                time.sleep(1)
            except Exception as e:
                print("Error processing post:", e)
    
    # Défilement
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
    # Vérifier si la page a été défilée jusqu'au bout
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        scroll_attempts += 1
        if scroll_attempts >= 3:  # Arrêter après 3 tentatives sans nouveaux éléments
            break
    else:
        scroll_attempts = 0  # réinitialiser le compteur si de nouveaux posts sont chargés
    last_height = new_height

# Affichage ou sauvegarde des données
print(posts)

In [ ]:
post_df =pd.DataFrame(posts)
post_df

# Extraction dans un csv

In [ ]:
post_df.to_csv('insta_v0.csv', index=False)  